We will need to analyse the data using seaborn and matplotlib to visualise some main areas.
    -missing data
    -duplicates
    -any strange variables and outliers
    -formats of data

we will use these options to visualise the areas we have listed. 
    -visually check the data to see if there are any missing values
    -check the names against eachother to see if there are any duplicates 
    -use a boxplot to check for outliers and to see if the datasets have a squew
    -check the units of the columns are equal 


visual analysis 

we can see that missing data is left blank. this makes our checks for data easy however there are multiple rows that have more then one missing pieces of data these will have to be analised and a descision will have to be made on what to do with these pieces of data.



In [1]:
#main imports of the general modules
import xlsxwriter


import pandas as pd
import numpy as np

In [2]:
od1 = pd.read_excel("od1.xlsx")
od2 = pd.read_excel("od2.xlsx")
os = pd.read_excel("os.xlsx")


In [3]:
print(od1.columns)
print(od2.columns)
print(os.columns)

Index(['ID', 'Age', 'Gender', 'Diagnosis', 'dioptre_1', 'dioptre_2',
       'astigmatism', 'Pachymetry'],
      dtype='object')
Index(['Diagnosis', 'dioptre_1', 'dioptre_2', 'astigmatism',
       'Phakic/Pseudophakic', 'Pneumatic', 'Perkins', 'Pachymetry',
       'Axial_Length', 'VF_MD'],
      dtype='object')
Index(['Unnamed: 0', 'Age', 'Gender', 'Diagnosis', 'Refractive_Defect',
       'Unnamed: 5', 'Unnamed: 6', 'Phakic/Pseudophakic', 'IOP', 'Unnamed: 9',
       'Pachymetry', 'Axial_Length', 'VF_MD'],
      dtype='object')


let's merge the datasets of od1 and od2 so we can have a functioning dataset with all the columns required


In [4]:
def get_merged_header_ranges(header):
    ranges = []
    current_value = header[0]
    start_col = 0
    
    for col_num in range(1, len(header)):
        if header[col_num] != current_value:
            ranges.append((current_value, start_col, col_num - 1))
            current_value = header[col_num]
            start_col = col_num
            
    ranges.append((current_value, start_col, col_num))
    return ranges


In [5]:
od = pd.merge(od1,od2, how= 'inner', on=list(set(od1.columns) & set(od2.columns)))

In [6]:
print(od)

       ID  Age  Gender   Diagnosis  dioptre_1  dioptre_2  astigmatism  \
0    #038   34  female     healthy       0.50      -1.00         90.0   
1    #280   60    male     healthy      -0.50      -1.00         23.0   
2    #198   67    male     healthy       0.50      -0.75        156.0   
3    #041   50  female  suspicious       1.75      -0.50        120.0   
4    #271   41    male     healthy      -1.25      -0.50        150.0   
..    ...  ...     ...         ...        ...        ...          ...   
239  #089   65  female  suspicious      -6.00      -0.25         10.0   
240  #264   61    male  suspicious       2.00      -0.75         90.0   
241  #056   65    male    glaucoma      -1.00      -2.75        100.0   
242  #018   43    male  suspicious      -2.00      -0.75         97.0   
243  #174   63  female     healthy      -3.25      -0.50         72.0   

     Pachymetry                 Phakic/Pseudophakic  Pneumatic  Perkins  \
0         599.0                                 

In [7]:
# Creating a multilevel DataFrame
header = pd.MultiIndex.from_tuples([
    ('', 'ID'),
    ('Age', 'Age'),
    ('Gender', 'Gender'),
    ('Diagnosis', 'Diagnosis'),
    ('Refractive_Defect', 'dioptre_1'),
    ('Refractive_Defect', 'dioptre_2'),
    ('Refractive_Defect', 'astigmatism'),
    ('Phakic/Pseudophakic', 'Phakic/Pseudophakic'),
    ('IOP', 'Pneumatic'),
    ('IOP', 'Perkins'),
    ('Pachymetry', 'Pachymetry'),
    ('Axial_Length', 'Axial_Length'),
    ('VF_MD', 'VF_MD')
])
reorder = ['ID','Age','Gender','Diagnosis','dioptre_1','dioptre_2','astigmatism','Phakic/Pseudophakic','Pneumatic','Perkins','Pachymetry','Axial_Length','VF_MD']
od = od[reorder]




In [8]:

with pd.ExcelWriter('od.xlsx', engine='xlsxwriter') as writer:
    # Write the DataFrame to the Excel file
    od.to_excel(writer, index=False, sheet_name='Sheet1')

    # Get the xlsxwriter workbook and worksheet objects
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    # Apply formatting to the first-level header row
    header_format = workbook.add_format({'bold': True,  'align': 'center'})
    
    merged_header_ranges = get_merged_header_ranges(header.get_level_values(0))
    for value, start_col, end_col in merged_header_ranges:
        if start_col == end_col:
            worksheet.write(0, start_col, value, header_format)
        else:
            worksheet.merge_range(0, start_col, 0, end_col, value, header_format)

    # Apply formatting to the second-level header row
    for col_num, value in enumerate(header.get_level_values(1)):
        worksheet.write(1, col_num, value, header_format)

    # Set column widths
    worksheet.set_column(0, len(header.get_level_values(1)) - 1, 15)


This is the end of part 1